In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load the saved XGBoost model
model_path = '/path/to/the/model.pkl'  # path to your saved model
best_xgb_model = joblib.load(model_path)

# Load the preprocessor object (assuming it was saved as 'preprocessor.pkl')
preprocessor_path = '/path/to/the/preprocessor.pkl'  # path to your saved preprocessor
preprocessor = joblib.load(preprocessor_path)  # Load the preprocessor

def preprocess_and_predict(age, income, loanamount, creditscore, monthsemployed, numcreditlines,
                           interestrate, loanterm_months, dtiratio, education, employmenttype,
                           maritalstatus, hasmortgage, hasdependents, loanpurpose, hascosigner):
    # Calculate total payment
    totalpayment = loanamount * (1 + interestrate / 100 * loanterm_months / 12)

    # Prepare input data as DataFrame
    data = {
        'age': [age], 'income': [income], 'loanamount': [loanamount], 'creditscore': [creditscore],
        'monthsemployed': [monthsemployed], 'numcreditlines': [numcreditlines], 'interestrate': [interestrate],
        'loanterm': [loanterm_months], 'dtiratio': [dtiratio], 'education': [education],
        'employmenttype': [employmenttype], 'maritalstatus': [maritalstatus],
        'hasmortgage': ['Yes' if hasmortgage else 'No'], 'hasdependents': ['Yes' if hasdependents else 'No'],
        'loanpurpose': [loanpurpose], 'hascosigner': ['Yes' if hascosigner else 'No'], 'totalpayment': [totalpayment]
    }
    df = pd.DataFrame(data)

    # Apply preprocessor transformations
    df_prepared = preprocessor.transform(df)

    # # Check the prepared DataFrame
    # print("Prepared DataFrame:")
    # print(df_prepared)

    # Predict using the model
    probabilities = best_xgb_model.predict_proba(df_prepared)[:, 1]
    prediction = best_xgb_model.predict(df_prepared)[0]

    return prediction, probabilities[0]


def interactive_predict():
    print("Welcome to Loan Default Prediction System")
    age = int(input("Input Age: "))
    income = float(input("Input Income: "))
    loanamount = float(input("Input Loan Amount: "))
    creditscore = int(input("Input Credit Score: "))
    monthsemployed = int(input("Input Months Employed: "))
    numcreditlines = int(input("Input Number of Credit Lines: "))
    interestrate = float(input("Input Interest Rate (%): "))
    loanterm_months = int(input("Input Loan Term (Months): "))
    dtiratio = float(input("Input Debt-to-Income Ratio (%): "))
    education = input("Input Education (e.g., Master's): ")
    employmenttype = input("Input Employment Type (e.g., Full-time): ")
    maritalstatus = input("Input Marital Status (e.g., Single): ")
    hasmortgage = input("Has Mortgage (Yes/No): ").lower() == 'yes'
    hasdependents = input("Has Dependents (Yes/No): ").lower() == 'yes'
    loanpurpose = input("Input Loan Purpose: ")
    hascosigner = input("Has Cosigner (Yes/No): ").lower() == 'yes'

    # Predict using the function
    prediction, probability = preprocess_and_predict(age, income, loanamount, creditscore, monthsemployed, numcreditlines,
                                                     interestrate, loanterm_months, dtiratio, education, employmenttype,
                                                     maritalstatus, hasmortgage, hasdependents, loanpurpose, hascosigner)

    # Print prediction and probability
    print(f"\nPrediction: {'Defaulter' if prediction == 1 else 'Non-defaulter'}")
    print(f"Probability of being a defaulter: {probability:.4f}")

# Run interactive prediction
interactive_predict()